1. Which site has the most successful launches?

KSC LC-39A has the most successful launches with 41.7% of all successful launches as shown in the pie chart for All
Sites.

2. Which site has the highest launch success rate?

CCAFS SLC-40 has the highest success rate, with a success rate of 42.9%. This can be seen by looking at the pie chart
for each site individually.

3. Which payload range(s) has the highest launch success rate?

A quick glance at the scatter plot indicates that 2000kg-4000kg and 4500kg-5500kg are the payload ranges with the
highest success rates.

4. Which payload range(s) has the lowest launch success rate?

This is complimentary to the previous answer. The ranges <=2000kg, 4000kg-4500kg, and >=6000kg are the ranges with the
lowest success rate.

5. Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?

Technically the B5 booster version has the highest launch success rate, however it has a sample size of 1 meaning that
this data is unreliable.

Of the others FT booster version has the highest launch success rate. Perhaps followed by/competing with the B4 booster
version. Conversely, v1.0 and v1.1 barely have any successful launches.

In [42]:
%pip install numpy
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [43]:
import numpy as np
import pandas as pd

In [4]:
spacex_df = pd.read_csv('./spacex_launch_dash.csv')
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [31]:
# Create success metrics based on booster version
booster_df = spacex_df[['Booster Version Category']].groupby(['Booster Version Category'], as_index=False).first()
booster_df['Number Successes'] = [len(spacex_df[(spacex_df['Booster Version Category'] == booster_df.iloc[i]['Booster Version Category'])
                                                & (spacex_df['class'] == 1)])
                              for i in range(len(booster_df))]
booster_df['Number Launches'] = [len(spacex_df[spacex_df['Booster Version Category'] == booster_df.iloc[i]['Booster Version Category']])
                              for i in range(len(booster_df))]
booster_df['Success Rate'] = booster_df['Number Successes']/booster_df['Number Launches']
booster_df

,Booster Version Category,Number Successes,Number Launches,Success Rate
0,B4,6,11,0.545455
1,B5,1,1,1.000000
2,FT,16,24,0.666667
3,v1.0,0,5,0.000000
4,v1.1,1,15,0.066667


The numbers confirm my answer to question 5.

In [54]:
range_min = 0
range_max = 10000
step = 1000

payload_range_success = pd.DataFrame()
payload_range_success['Range'] = []
payload_range_success['Success Rate'] = []
payload_range_success['Number Launches'] = []

for min_payload in range(range_min, range_max + step, step):
    for max_payload in range(min_payload, range_max + step, step):
        filtered_df = spacex_df[['class', 'Payload Mass (kg)']][(spacex_df['Payload Mass (kg)']>= min_payload)
                                                                & (spacex_df['Payload Mass (kg)']<= max_payload)]
        number_launches = len(filtered_df)
        if number_launches > 0: 
            success_rate = float(filtered_df['class'].mean())
            payload_range_success.loc[len(payload_range_success.index)] = [(min_payload, max_payload), success_rate, number_launches]

payload_range_success.sort_values(by='Success Rate',ascending=False, inplace=True)
display(payload_range_success.head(10))
display(payload_range_success.tail(10))
        

,Range,Success Rate,Number Launches
28,"(3000, 4000)",0.727273,11
21,"(2000, 4000)",0.619048,21
52,"(9000, 10000)",0.600000,5
51,"(8000, 10000)",0.600000,5
50,"(7000, 10000)",0.600000,5
13,"(1000, 4000)",0.583333,24
29,"(3000, 5000)",0.578947,19
22,"(2000, 5000)",0.551724,29
30,"(3000, 6000)",0.541667,24
14,"(1000, 5000)",0.531250,32


,Range,Success Rate,Number Launches
39,"(4000, 9000)",0.294118,17
2,"(0, 2000)",0.230769,13
44,"(5000, 9000)",0.222222,9
43,"(5000, 8000)",0.222222,9
42,"(5000, 7000)",0.222222,9
1,"(0, 1000)",0.200000,10
46,"(6000, 7000)",0.000000,4
47,"(6000, 8000)",0.000000,4
48,"(6000, 9000)",0.000000,4
0,"(0, 0)",0.000000,2


The numbers here somewhat corroborate my answers to 3 and 4. However, with this analysis there is a bit of redundancy.
For instance, we have nested data windows here. Such as the (5000,9000) range being contained in the (4000,9000) range
which has a higher success rate. Furthermore, we also have nested ranges where all of them have the same success rate.
Collapsing the ranges based on containment and similar success rate might capture this success data better.